In [ ]:
!pip install selenium
!apt install chromium-chromedriver

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import matplotlib.pyplot as plt
import re
import sqlite3

conn = sqlite3.connect('mydatabase.db')
print("Opened database successfully")
cur = conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS place (descripcion  CHAR(50))")
cur.execute("CREATE TABLE IF NOT EXISTS flight (origen INTEGER, destino INTEGER, compania CHAR(50), escalaida INTEGER, tiempoida REAL, escalavuelta INTEGER, tiempovuelta INTEGER ,price REAL)")

cur.execute("INSERT INTO place(descripcion) VALUES('BUENOS-AIRES')")
conn.commit()

website = 'https://www.despegar.com.ar/shop/flights/results/roundtrip/BUE/MAD/2022-12-01/2022-12-15/1/0/0/NA/NA/NA/NA/NA?from=SB&di=1-0#showModal'

SCROLL_PAUSE_TIME = 0.5

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')

driver = webdriver.Chrome('chromedriver', options=options)
driver.get(website)

last_height = driver.execute_script("return document.body.scrollHeight")

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'COMMON')))
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

name = soup.find_all('cluster', class_= 'COMMON')

for link in name:
    compania_1 = link.find('span', class_='name').get_text().lstrip()
    price = link.find('span', class_='amount price-amount').get_text()
    way = link.find_all('span', class_='sub-cluster');
    escalas = way[0].find('span', class_='stops-text').get_text()
    escalas = re.findall('\d+.\d*|', escalas)
    escalas= escalas[1]
    escalas = str(escalas)
    #print(escalas)
    
    fly_duration = way[0].find('span', class_='best-duration').get_text()
    fly_duration = re.findall('\d+.\d*|', fly_duration)
    fly_duration = list(fly_duration)
    
    for i in fly_duration:
        if i == '':
            fly_duration.remove(i)
        for g in fly_duration[::-1]:
            if g == '':
                fly_duration.remove(g)
        
    vuelta = way[1].find('span', class_='stops-wrapper text').get_text()
    escalas_vuelta = re.findall('\d+.\d*|', vuelta)
    comeback_scales = escalas_vuelta[1]
    comeback_scales = str(comeback_scales)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 5 not upgraded.
Need to get 95.1 MB of archives.
After this operation, 319 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 107.0.5304.87-0ubuntu11.18.04.1 [1,158 kB]
Get:2 http://archi